In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datasets
import torch
import random
import json
from tqdm.auto import tqdm
import pandas as pd
from pathlib import Path

import transformers

from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [4]:
# assumes PYTHONPATH is pointing to the root of the repo
from codet5_finetune.options import options 
from codet5_finetune.data import DataCollatorNTP
from codet5_finetune.train import prepare

In [5]:
# assumes current folder is <repo_root>/dev
cwd =!pwd
cwd = Path(cwd[0])
opt = options({'common_config': cwd.parent / 'codet5_finetune/common_config.yaml'})
opt.dataloader_num_workers = 0

['/home/toolkit/.conda/envs/rlctx_p39/lib/python3.9/site-packages/ipykernel_launcher.py', '-f', '/home/toolkit/.local/share/jupyter/runtime/kernel-366b5caa-e1cb-41ac-8e78-7440ef5176c9.json']


In [6]:
opt.path_java_filtered_subset

'/repo_data/the_stack11_dedup_alt_comments_no_1K_set_subset/data'

In [7]:
ctx = prepare(opt)

opt.per_device_train_batch_size=8
len(ctx.ds["train"])=300000


loading configuration file config.json from cache at /home/toolkit/.cache/huggingface/hub/models--Salesforce--codet5-base/snapshots/4078456db09ba972a3532827a0b5df4da172323c/config.json
Model config T5Config {
  "_name_or_path": "Salesforce/codet5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "bos_token_id": 1,
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 2,
  "feed_forward_proj": "relu",
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_para

In [10]:
res = ctx.trainer.evaluate(ctx.ds['test'].select(range(1000)))

***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
Generate config GenerationConfig {
  "bos_token_id": 1,
  "decoder_start_token_id": 0,
  "eos_token_id": 2,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 1,
  "decoder_start_token_id": 0,
  "eos_token_id": 2,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 1,
  "decoder_start_token_id": 0,
  "eos_token_id": 2,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 1,
  "decoder_start_token_id": 0,
  "eos_token_id": 2,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 1,
  "decoder_start_token_id": 0,
  "eos_token_id": 2,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 1,
  "decoder_start_token_id": 0,
  "eos_t

len(labels)=1000


In [12]:
res

{'eval_loss': 7.382516860961914,
 'eval_em': 1000,
 'eval_num': [1000],
 'eval_runtime': 51.6851,
 'eval_samples_per_second': 19.348,
 'eval_steps_per_second': 2.418}

In [ ]:
import numpy as np
label_ids = np.where(ctx.preds.label_ids != -100, label_ids, ctx.tokenizer.pad_token_id)
labels = ctx.tokenizer.batch_decode(label_ids, skip_special_tokens=True)
predictions = ctx.tokenizer.batch_decode(ctx.preds.predictions, skip_special_tokens=True)
res = {
    'em': sum(el_pred == el_label for el_pred, el_label in zip(predictions, labels))
}

In [ ]:
res

In [ ]:
tokenizer.decode(preds.inputs[7])

In [ ]:
preds.label_ids[preds.label_ids == -100] = 0
tokenizer.decode(preds.label_ids[7])

In [ ]:
tokenizer.decode(preds.predictions, skip_special_tokens=False)

In [ ]:
tokenizer.batch_decode(preds.predictions, skip_special_tokens=True)

In [ ]:
tokenizer.all_special_tokens_extended['<extra_id_0>']

In [ ]:
tokenizer.vocab['<extra_id_0>']

In [ ]:
type(preds.predictions)